This notebook is a tutorial that demonstrates how to configure a questionnaire simulation using Concordia.

# <a href="https://colab.research.google.com/github/google-deepmind/concordia/blob/main/examples/questionnaire_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Colab-specific setup (use a CodeSpace to avoid the need for this).
try:
  %env COLAB_RELEASE_TAG
except:
  pass  # Not running in colab.
else:
  %pip install --ignore-requires-python --requirement 'https://raw.githubusercontent.com/google-deepmind/concordia/main/examples/requirements.in' 'git+https://github.com/google-deepmind/concordia.git#egg=gdm-concordia'
  %pip list

In [ ]:
# @title Imports

import numpy as np
from IPython import display

import sentence_transformers

from concordia.language_model import gpt_model
from concordia.language_model import no_language_model

from concordia.typing import prefab as prefab_lib
import concordia.prefabs.simulation.questionnaire_simulation as questionnaire_simulation
import concordia.prefabs.entity as entity_prefabs
import concordia.prefabs.game_master as game_master_prefabs

from concordia.utils import helper_functions

In [ ]:
# @title Language Model - pick your model and provide keys or select DISABLE_LANGUAGE_MODEL

# By default this colab uses GPT-4, so you must provide an API key.
# Note that it is also possible to use local models or other API models,
# simply replace this cell with the correct initialization for the model
# you want to use.
GPT_API_KEY = '' #@param {type: 'string'}
GPT_MODEL_NAME = 'gpt-4.1-nano' #@param {type: 'string'}

if not GPT_API_KEY:
  raise ValueError('GPT_API_KEY is required.')

# To debug without spending money on API calls, set DISABLE_LANGUAGE_MODEL=True
DISABLE_LANGUAGE_MODEL = False

if not DISABLE_LANGUAGE_MODEL:
  model = gpt_model.GptLanguageModel(api_key=GPT_API_KEY,
                                     model_name=GPT_MODEL_NAME)
else:
  model = no_language_model.NoLanguageModel()


## Language Model setup

In [ ]:
# @title Setup sentence encoder

if DISABLE_LANGUAGE_MODEL:
  embedder = np.ones(3)
else:
  st_model = sentence_transformers.SentenceTransformer(
      'sentence-transformers/all-mpnet-base-v2')
  embedder = lambda x: st_model.encode(x, show_progress_bar=False)

In [ ]:
test = model.sample_text(
    'Is societal and technological progress like getting a clearer picture of '
    'something true and deep?')
print(test)

# Parameters

In [ ]:
# @title Load prefabs from packages to make the specific palette to use here.

prefabs = {
    **helper_functions.get_package_classes(entity_prefabs),
    **helper_functions.get_package_classes(game_master_prefabs),
}

In [ ]:
display.display(
    display.Markdown(helper_functions.print_pretty_prefabs(prefabs)))

In [ ]:
from concordia.contrib.data.questionnaires import depression_stress_anxiety_scale

DASS = depression_stress_anxiety_scale.DASSQuestionnaire()


In [ ]:
lord_of_the_rings_characters = [
    "Frodo Baggins",
    "Samwise Gamgee",
    "Meriadoc Brandybuck",
    "Peregrin Took",
    "Gandalf",
    "Aragorn",
    "Legolas",
    "Gimli",
    "Boromir",
    "Sauron",
    "Saruman",
    "Gollum",
    "Arwen",
    "Galadriel",
    "Elrond",
    "Théoden",
    "Éowyn",
    "Faramir",
    "Denethor",
    "Bilbo Baggins"
]

In [ ]:
# @title Scene 1 - Frodo in the Shire

instances = []

for name in lord_of_the_rings_characters:
  instances.append(prefab_lib.InstanceConfig(
      prefab='basic__Entity',
      role=prefab_lib.Role.ENTITY,
      params={
          'name': name,
      },
  ))

instances.append(prefab_lib.InstanceConfig(
        prefab='interviewer__GameMaster',
        role=prefab_lib.Role.GAME_MASTER,
        params={
            'name': 'interviewer',
            'player_names': lord_of_the_rings_characters,
            'questionnaires': [DASS],
            'verbose': False,
            'embedder': embedder,
        },
    ),

)

In [ ]:
config = prefab_lib.Config(
    default_premise=f'Frodo is making his way to mount Doom, while Sauron is searching for the Ring.',
    default_max_steps=1,
    prefabs=prefabs,
    instances=instances,
)

In [ ]:
# @title Initialize the simulation
runnable_simulation = questionnaire_simulation.QuestionnaireSimulation(
    config=config,
    model=model,
    embedder=embedder,
)

In [ ]:
# @title Run the simulation
results_log = runnable_simulation.play(max_steps=1)

In [ ]:
questionnaire_comp = runnable_simulation.game_masters[0].get_component(
    'questionnaire'
)
questionnaire_result_df = questionnaire_comp.get_questionnaires_results()
print(questionnaire_result_df)

In [ ]:
print(questionnaire_comp.get_answers())

```
Copyright 2025 DeepMind Technologies Limited.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```